In [25]:
import random
import itertools
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt

In [26]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime(2025, 2, 24)

tickers = ['AAPL','MSFT','F','BAC','NKE','V','DIS']

n_tickers_selected = 4
n_combinations = 20

sampled_combinations = random.sample(list(itertools.combinations(tickers, n_tickers_selected)), n_combinations)

In [55]:
all_porffolios = []

for combination in sampled_combinations:
    data = yf.download(list(combination), start= start_date, end = end_date, progress= False)["Close"]
    #print(data.head())
    returns = data.pct_change().dropna()
    #print(returns.head())
    covariance_matrix = returns.cov()
    #print(covariance_matrix)

    annual_returns = returns.mean() * 252
    annual_cov = covariance_matrix.cov() * 252

    # Numero de simulaciones de ponderaciones
    num_portafolios = 1000

    weights_array = np.zeros((num_portafolios, n_tickers_selected))

    returns_array = np.zeros(num_portafolios)
    risk_array = np.zeros(num_portafolios)

    for i in range(num_portafolios):
        # generas los pesos aleatorios para cada portafolio
        weights_array[i] = np.random.random(n_tickers_selected)
        weights_array[i] = weights_array[i] / np.sum(weights_array[i])

        # obteniendo rendimiento y volatilidad esperada anual por portafolio
        portfolio_return = np.dot(annual_returns, weights_array[i])
        portfolio_risk = np.sqrt(np.dot(weights_array[i].T, np.dot(annual_cov, weights_array[i])))

        # guardando historicos
        weights_array[i, :] = weights_array[i]
        returns_array[i] = portfolio_return
        risk_array[i] = portfolio_risk


    #print(returns_array)
    #print(risk_array)
    portfolios = pd.DataFrame({
        "Returns": returns_array,
        "Risk": risk_array
    })


    for i, symbol in enumerate(combination):
        portfolios[symbol + " Weight"] = weights_array[:, i]

    all_porffolios.append(portfolios)

all_porffolios[0].head()

,Returns,Risk,AAPL Weight,MSFT Weight,F Weight,NKE Weight
0,-0.011961,0.000928,0.233113,0.321777,0.260541,0.184569
1,-0.027302,0.000990,0.166099,0.324030,0.279650,0.230222
2,-0.080816,0.001239,0.197120,0.200135,0.075942,0.526803
3,-0.001687,0.000969,0.416353,0.327800,0.094151,0.161696
4,0.008059,0.001269,0.498270,0.408667,0.025415,0.067648
